In [1]:
import time 
import numpy as np
from pyvisa import VisaIOError, ResourceManager
import pylabnet.network.client_server.tektronix_awg70k as tektronix_awg70k
from pylabnet.hardware.awg.tektronix_awg70k import Tek70K
client = tektronix_awg70k.Client(
    host='192.168.50.114', 
    port=29618
)
# rm = ResourceManager()
# my_instrument = rm.open_resource('TCPIP0::192.168.50.207::INSTR')
tek = Tek70K(client)

c:\Users\Deagol\pylabnet\env\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def initialize_AWG(tek, sample_rate, awg_max_amp, N_markers):
    tek.reset()
    time.sleep(2)
    tek.wait()

    ## Output
    tek.write('OUTPUT:PATH DIRECT')
    tek.write(f'CLOCK:SRATE {sample_rate}')
    tek.write(f'VOLTAGE:LEVEL:AMPL {awg_max_amp}')
    # Max 10 bit resolution, 1 bit used per marker 
    tek.write(f'DAC:RESOLUTION {10-N_markers}') 

    ## Waveforms
    tek.clear_all_waveforms()

    ## Sequences
    tek.clear_all_sequences()
    tek.wait()

def write_wait_sequence(tek, seq, wait_samp=4800):
    tek.change_plugin("Basic Waveform")
    seq_name, step_ptr = seq.name, seq.step_ptr

    wait_wave_name = "WAIT_WAVE"

    tek.write(f'BWAV:LENGTH {wait_samp}') 
    tek.write('BWAV:AMPL 0') 
    tek.write(f'BWAV:COMPile:NAME "{wait_wave_name}"') 
    tek.write('BWAV:COMPILE') # Compile
    tek.wait()

    # Add a step
    tek.write(f'SLIST:SEQ:STEP:ADD "{seq_name}", {step_ptr}, 1')
    # Assign the wait waveform to Track 1
    tek.write(f'SLIST:SEQ:STEP{step_ptr}:TASSET1:WAVEFORM "{seq_name}", "{wait_wave_name}"')
    # Make it repeat infinitely
    tek.write(f'SLIST:SEQ:STEP{step_ptr}:RCOUNT "{seq_name}", INF')
    # Add wave table entry
    seq.wave_table[wait_wave_name] = seq.step_ptr

    seq.step_ptr += 1
    return seq

In [3]:
def write_rabi_waves(tek, sample_rate, awg_max_amp, pulse_freq, t_rabi, N, min_amp, max_amp):
    tek.change_plugin("Basic Waveform")
    tek.write(f'BWAV:SRATE {sample_rate}') # Sample rate
    tek.write('BWAV:FDR OFF') # Full dynamic range?
    tek.write('BWAV:FUNC SINE') # Waveform Shape
    tek.write(f'BWAV:FREQ {pulse_freq}') # Frequency
    tek.write(f'BWAV:LENGTH {t_rabi * sample_rate}') # Number of samples

    assert(max_amp <= awg_max_amp)
    amps = np.linspace(min_amp, max_amp, N)

    # Generate Rabi waveforms
    for i in range(N):
        tek.write(f'BWAV:AMPL {amps[i]}') 
        tek.write(f'BWAV:COMPile:NAME "Rabi_{i}"') # Name waveform
        tek.write('BWAV:COMPILE') # Compile
        tek.wait()
        tek.write_marker(f"Rabi_{i}", [1 for _ in range(1000)])
        tek.wait()
        
def write_rabi_sequence(tek, seq, N):
    seq_name, step_ptr = seq.name, seq.step_ptr
   
    for i in range(0, N):
        # Add a step for each Rabi waveform 
        tek.write(f'SLIST:SEQ:STEP:ADD "{seq_name}", {step_ptr+i}, 1')
        # Assign the Rabi waveform to Track 1
        tek.write(f'SLIST:SEQ:STEP{step_ptr+i}:TASSET1:WAVEFORM "{seq_name}", "Rabi_{i}"')
        # Make the wave wait for Trigger A
        tek.write(f'SLIST:SEQ:STEP{step_ptr+i}:WINPUT "{seq_name}", ATR')
        # Add wave table entry
        seq.wave_table[f"Rabi_{i}"] = step_ptr + i

    # Make the last step go to first
    tek.write(f'SLIST:SEQ:STEP{step_ptr+(N-1)}:GOTO "{seq_name}", FIRST')

    seq.step_ptr += N # Advance the step pointer
    return seq

In [4]:
t_rabi = 10E-6
freq = 100E6
pts = 5
min_amp, max_amp = 0.1, 0.5
marker_len = 3000

SAMPLE_RATE = 50E9 # Max 50 GSa/s
AWG_MAX_AMP = 0.5 # 0.3 to 0.5 Vpp
N_MARKERS = 1 # 0, 1 or 2 markers
# clear_all_waveforms(self):
initialize_AWG(tek, SAMPLE_RATE , AWG_MAX_AMP, N_MARKERS)
write_rabi_waves(tek, SAMPLE_RATE, AWG_MAX_AMP, freq, t_rabi, pts, min_amp, max_amp)

seq = tek.create_new_sequence("Hi")
seq = write_rabi_sequence(tek, seq, pts)
tek.assign_sequence(seq.name)

In [ ]:
# tek.write_marker("Rabi_1", [1 for _ in range(marker_len)])
# tek.assign_waveform("Rabi_1")

In [6]:
# tek.awg_run()
# tek.set_output_state(1)
# tek.awg_stop()
# tek.set_output_state(0)

In [7]:
# tek.reset()

In [12]:
# tek.write('MMEM:MSIS "C:"')
# tek.write('MMEM:CDIR "C:\ProgramData\Tektronix\AWG\AWG70k\EquationEditor"')
print(tek.get_dir())
# tek.change_dir("C", "Users")
# print(tek.get_dir())
# tek.write('AWGCONTROL:COMP "C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\Untitled.equ"')

"\"



In [5]:
a = bytearray("""clock = 10e9 'Recommended sample rate
size = 2400
"Square2pt5"=sqr(2.5 * 2 * pi * scale) * -1
"Square2pt5".Marker1 = (point= 480)
"Square2pt5".Marker1 = "Square2pt5".Marker1 + (point= 960)
"Square2pt5".Marker2 = (point <= 5)""", encoding='ascii')

command ='heehee'

In [7]:
import pickle

b = pickle.dumps([command, a])
c = pickle.loads(b)

In [8]:
c[0]

'heehee'

In [9]:
isinstance(c, bytearray)

False

In [24]:
tek.write_file("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\abc.equ",
"""clock = 10e9 'Recommended sample rate
size = 2400 
"Square2pt5"=sqr(2.5 * 2 * pi * scale) * -1
"Square2pt5".Marker1 = (point= 480)
"Square2pt5".Marker1 = "Square2pt5".Marker1 + (point= 960)
"Square2pt5".Marker2 = (point <= 5)""")

In [25]:
print(tek.read_file("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\abc.equ"))
# print(tek.get_file_size("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\xy8_waveforms.equ"))

clock = 10e9 'Recommended sample rate
size = 2400 
"Square2pt5"=sqr(2.5 * 2 * pi * scale) * -1
"Square2pt5".Marker1 = (point= 480)
"Square2pt5".Marker1 = "Square2pt5".Marker1 + (point= 960)
"Square2pt5".Marker2 = (point <= 5)


In [26]:
tek.compile_equation("C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\abc.equ")

In [6]:
from pyvisa import VisaIOError, ResourceManager

rm = ResourceManager()

device = rm.open_resource("TCPIP0::192.168.50.207::INSTR")
device_id = device.query('*IDN?')

In [7]:
device_id

'TEKTRONIX,AWG70001B,B020539,FV:8.1.0266.0\n'

In [8]:
path="C:\\ProgramData\\Tektronix\\AWG\\AWG70k\\EquationEditor\\hij.equ"


string="""clock = 10e9 'Recommended sample rate
size = 2400
"Square2pt5"=sqr(2.5 * 2 * pi * scale) * -1
"Square2pt5".Marker1 = (point= 480)
"Square2pt5".Marker1 = "Square2pt5".Marker1 + (point= 960)
"Square2pt5".Marker2 = (point <= 5)"""
start_index=0

In [9]:
device.write_binary_values(f'MMEMORY:DATA "{path}", {start_index}, ', bytearray(string, encoding='ascii'), datatype='b')

309